# Reshape by ACS table

In [1]:
import numpy as np
import pandas as pd
import intake
import os

In [2]:
df = pd.read_parquet('../data/raw_census.parquet')

In [19]:
def make_percents(row):
    emp_cond = (row.table=='emp') and (row.main_var in ['lf16', 'emp16', 'unempr16'])
    income_cond = (row.table=='income') and (row.main_var=='hh') and (row.second_var != 'hh')
    edu_cond = (row.table=='edu') and (row.second_var != 'pop25') and (row.second_var.find('medearning') == -1)
    food_cond = (row.table=='food') and (row.second_var != 'hh') and (row.second_var.find('income') == -1)
    if emp_cond or edu_cond or income_cond:
        return (row.value/100)
    elif (row.table=='aggpubassist') and (row.value < -100000):
        return np.nan
    else:
        return row.value

df['value2'] = df.apply(make_percents, axis = 1)

In [6]:
povfam = df[df.table=='povfam']
# Need to re-download C02 column

In [7]:
povfam

,GEOID,year,variable,value,table,main_var,second_var,new_var
336,06037141400,2010,S1702_C01_001E,1219.0,povfam,families,families_pov,families_families_pov
2683,06037141400,2012,S1702_C01_001E,1204.0,povfam,families,families_pov,families_families_pov
5029,06037141400,2013,S1702_C01_001E,1181.0,povfam,families,families_pov,families_families_pov
7375,06037141400,2014,S1702_C01_001E,1069.0,povfam,families,families_pov,families_families_pov
9399,06037141400,2015,S1702_C01_001E,1079.0,povfam,families,families_pov,families_families_pov
11850,06037141400,2016,S1702_C01_001E,1130.0,povfam,families,families_pov,families_families_pov
14966,06037141400,2017,S1702_C01_001E,1242.0,povfam,families,families_pov,families_families_pov
16758,06037141400,2011,S1702_C01_001E,1247.0,povfam,families,families_pov,families_families_pov


In [ ]:
pubassist = df[df.table=='pubassist']
# These are number of families that received food stamps
# Need to derive percents

In [ ]:
pubassist